### Question:
Generate the output as shown below, where for First_name, Second_name and Last_name columns will be generated by utilizing UDFs.

| DATE       | TITLE         | first_name | last_name | middle_name |
| ---------- | ------------- | ---------- | --------- | ----------- |
| 02-08-2017 | Councilmember | Jennifer   | Gates     | S.          |
| 02-08-2017 | Councilmember | Philip     | Kingston  | T.          |
| 02-08-2017 | Mayor         | Michael    | Rawlings  | S.          |
| 02-08-2017 | Councilmember | Adam       | Medrano   |             |
| 02-08-2017 | Councilmember | Casey      | Thomas    |             |
| 02-08-2017 | Councilmember | Carolyn    | Arnold    | King        |
| 02-08-2017 | Councilmember | Scott      | Griggs    |             |
| 02-08-2017 | Councilmember | B.         | McGough   | Adam        |
| 02-08-2017 | Councilmember | Lee        | Kleinman  |             |
| 02-08-2017 | Councilmember | Sandy      | Greyson   |             |
| 02-08-2017 | Councilmember | Jennifer   | Gates     | S.          |
| 02-08-2017 | Councilmember | Philip     | Kingston  | T.          |
| 02-08-2017 | Mayor         | Michael    | Rawlings  | S.          |
| 02-08-2017 | Councilmember | Adam       | Medrano   |             |
| 02-08-2017 | Councilmember | Casey      | Thomas    |             |
| 02-08-2017 | Councilmember | Carolyn    | Arnold    | King        |
| 02-08-2017 | Councilmember | Rickey     | Callahan  | D.          |
| 01-11-2017 | Councilmember | Jennifer   | Gates     | S.          |
| 04/25/2018 | Councilmember | Sandy      | Greyson   |             |
| 04/25/2018 | Councilmember | Jennifer   | Gates     | S.          |

In [10]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'

In [11]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

In [44]:
import pyspark.sql.functions as F
from pyspark.sql.types import *

# load the .csv file
voter_df = spark.read.format('csv').options(header=True)\
.load('file:///home/talentum/shared/Assignments/Evaluation_Labs/Lab2/DallasCouncilVoters.csv.gz')

voter_df.show()

+----------+-------------+-------------------+
|      DATE|        TITLE|         VOTER_NAME|
+----------+-------------+-------------------+
|02/08/2017|Councilmember|  Jennifer S. Gates|
|02/08/2017|Councilmember| Philip T. Kingston|
|02/08/2017|        Mayor|Michael S. Rawlings|
|02/08/2017|Councilmember|       Adam Medrano|
|02/08/2017|Councilmember|       Casey Thomas|
|02/08/2017|Councilmember|Carolyn King Arnold|
|02/08/2017|Councilmember|       Scott Griggs|
|02/08/2017|Councilmember|   B. Adam  McGough|
|02/08/2017|Councilmember|       Lee Kleinman|
|02/08/2017|Councilmember|      Sandy Greyson|
|02/08/2017|Councilmember|  Jennifer S. Gates|
|02/08/2017|Councilmember| Philip T. Kingston|
|02/08/2017|        Mayor|Michael S. Rawlings|
|02/08/2017|Councilmember|       Adam Medrano|
|02/08/2017|Councilmember|       Casey Thomas|
|02/08/2017|Councilmember|Carolyn King Arnold|
|02/08/2017|Councilmember| Rickey D. Callahan|
|01/11/2017|Councilmember|  Jennifer S. Gates|
|04/25/2018|C

In [45]:
# Add a new column called splits separated on whitespace
voter_df = voter_df.withColumn('splits', F.split(voter_df.VOTER_NAME, '\s+'))
# Filtering out rows where VOTER_NAME is null
voter_df = voter_df.filter(voter_df['VOTER_NAME'].isNotNull())
voter_df=voter_df.drop('VOTER_NAME')
voter_df.show()

+----------+-------------+--------------------+
|      DATE|        TITLE|              splits|
+----------+-------------+--------------------+
|02/08/2017|Councilmember|[Jennifer, S., Ga...|
|02/08/2017|Councilmember|[Philip, T., King...|
|02/08/2017|        Mayor|[Michael, S., Raw...|
|02/08/2017|Councilmember|     [Adam, Medrano]|
|02/08/2017|Councilmember|     [Casey, Thomas]|
|02/08/2017|Councilmember|[Carolyn, King, A...|
|02/08/2017|Councilmember|     [Scott, Griggs]|
|02/08/2017|Councilmember| [B., Adam, McGough]|
|02/08/2017|Councilmember|     [Lee, Kleinman]|
|02/08/2017|Councilmember|    [Sandy, Greyson]|
|02/08/2017|Councilmember|[Jennifer, S., Ga...|
|02/08/2017|Councilmember|[Philip, T., King...|
|02/08/2017|        Mayor|[Michael, S., Raw...|
|02/08/2017|Councilmember|     [Adam, Medrano]|
|02/08/2017|Councilmember|     [Casey, Thomas]|
|02/08/2017|Councilmember|[Carolyn, King, A...|
|02/08/2017|Councilmember|[Rickey, D., Call...|
|01/11/2017|Councilmember|[Jennifer, S.,

In [46]:
# Defining User-Defined Functions:

def getFirstName(names):
    if names[0] is None:
        return None
    return names[0]

def getSecondName(names):
    if names[1] is None:
        return None
    if len(names) > 2:
        return names[1]
    else:
        return None

def getLastName(names):
    if names[-1] is None:
        return None
    return names[-1]

# Register the UDFs with correct return type
udfFirstName = F.udf(getFirstName, StringType())
udfSecondName = F.udf(getSecondName, StringType())
udfLastName = F.udf(getLastName, StringType())

# Create new columns using the UDFs
voter_df = voter_df.withColumn('first_name', udfFirstName(F.col('splits')))
voter_df = voter_df.withColumn('second_name', udfSecondName(F.col('splits')))
voter_df = voter_df.withColumn('last_name', udfLastName(F.col('splits')))

voter_df = voter_df.drop('splits')

voter_df.show(truncate = False)

+----------+-------------+----------+-----------+---------+
|DATE      |TITLE        |first_name|second_name|last_name|
+----------+-------------+----------+-----------+---------+
|02/08/2017|Councilmember|Jennifer  |S.         |Gates    |
|02/08/2017|Councilmember|Philip    |T.         |Kingston |
|02/08/2017|Mayor        |Michael   |S.         |Rawlings |
|02/08/2017|Councilmember|Adam      |null       |Medrano  |
|02/08/2017|Councilmember|Casey     |null       |Thomas   |
|02/08/2017|Councilmember|Carolyn   |King       |Arnold   |
|02/08/2017|Councilmember|Scott     |null       |Griggs   |
|02/08/2017|Councilmember|B.        |Adam       |McGough  |
|02/08/2017|Councilmember|Lee       |null       |Kleinman |
|02/08/2017|Councilmember|Sandy     |null       |Greyson  |
|02/08/2017|Councilmember|Jennifer  |S.         |Gates    |
|02/08/2017|Councilmember|Philip    |T.         |Kingston |
|02/08/2017|Mayor        |Michael   |S.         |Rawlings |
|02/08/2017|Councilmember|Adam      |nul